## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
wcity_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
wcity_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Solone,UA,48.2081,34.8721,42.85,67,5,6.62,clear sky
1,1,Ribeira Grande,PT,38.5167,-28.7000,68.40,94,75,9.22,mist
2,2,Georgetown,MY,5.4112,100.3354,80.53,92,40,7.00,scattered clouds
3,3,Baruun-Urt,MN,46.6806,113.2792,29.88,35,100,16.78,snow
4,4,Vanavara,RU,60.3400,102.2797,28.06,67,100,8.16,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = wcity_data_df.loc[(wcity_data_df["Max Temp"] <= max_temp) & \
                                       (wcity_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ribeira Grande,PT,38.5167,-28.7000,68.40,94,75,9.22,mist
2,2,Georgetown,MY,5.4112,100.3354,80.53,92,40,7.00,scattered clouds
7,7,Vaini,TO,-21.2000,-175.2000,73.56,94,75,10.36,light intensity drizzle
8,8,Coquimbo,CL,-29.9533,-71.3436,62.33,67,40,13.80,scattered clouds
11,11,Mega,ET,4.0500,38.3000,61.88,71,13,6.78,few clouds
13,13,Faanui,PF,-16.4833,-151.7500,78.73,70,100,15.14,overcast clouds
15,15,Inhambane,MZ,-23.8650,35.3833,70.05,67,47,11.21,scattered clouds
18,18,Rikitea,PF,-23.1203,-134.9692,75.74,77,100,18.41,overcast clouds
21,21,Mount Isa,AU,-20.7333,139.5000,74.97,33,100,12.66,overcast clouds
24,24,Sabang,ID,5.8933,95.3214,77.05,84,100,13.02,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                1025
City                   1025
Country                1013
Lat                    1025
Lng                    1025
Max Temp               1025
Humidity               1025
Cloudiness             1025
Wind Speed             1025
Current Description    1025
dtype: int64

In [64]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [65]:
clean_df.count()

City_ID                1013
City                   1013
Country                1013
Lat                    1013
Lng                    1013
Max Temp               1013
Humidity               1013
Cloudiness             1013
Wind Speed             1013
Current Description    1013
dtype: int64

In [66]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ribeira Grande,PT,68.40,mist,38.5167,-28.7000,
2,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,
7,Vaini,TO,73.56,light intensity drizzle,-21.2000,-175.2000,
8,Coquimbo,CL,62.33,scattered clouds,-29.9533,-71.3436,
11,Mega,ET,61.88,few clouds,4.0500,38.3000,
13,Faanui,PF,78.73,overcast clouds,-16.4833,-151.7500,
15,Inhambane,MZ,70.05,scattered clouds,-23.8650,35.3833,
18,Rikitea,PF,75.74,overcast clouds,-23.1203,-134.9692,
21,Mount Isa,AU,74.97,overcast clouds,-20.7333,139.5000,
24,Sabang,ID,77.05,overcast clouds,5.8933,95.3214,


In [67]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [68]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ribeira Grande,PT,68.40,mist,38.5167,-28.7000,Quinta da Meia Eira
2,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,Cititel Penang
7,Vaini,TO,73.56,light intensity drizzle,-21.2000,-175.2000,Keleti Beach Resort
8,Coquimbo,CL,62.33,scattered clouds,-29.9533,-71.3436,Punto de encuentro
11,Mega,ET,61.88,few clouds,4.0500,38.3000,Amanuel Guest House
13,Faanui,PF,78.73,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
15,Inhambane,MZ,70.05,scattered clouds,-23.8650,35.3833,Oceano
18,Rikitea,PF,75.74,overcast clouds,-23.1203,-134.9692,People ThankYou
21,Mount Isa,AU,74.97,overcast clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
24,Sabang,ID,77.05,overcast clouds,5.8933,95.3214,Hotel Citra


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [76]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ribeira Grande,PT,68.40,mist,38.5167,-28.7000,Quinta da Meia Eira
2,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,Cititel Penang
7,Vaini,TO,73.56,light intensity drizzle,-21.2000,-175.2000,Keleti Beach Resort
8,Coquimbo,CL,62.33,scattered clouds,-29.9533,-71.3436,Punto de encuentro
11,Mega,ET,61.88,few clouds,4.0500,38.3000,Amanuel Guest House


In [77]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [78]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}
{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [79]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))